In [12]:
import pandas as pd
import numpy as np

In [ ]:
# Q1

In [3]:
df = pd.read_csv("results-gpt4o-mini.csv")

In [4]:
df = df.iloc[:300]

In [5]:
from sentence_transformers import SentenceTransformer
model_name="multi-qa-mpnet-base-dot-v1"
embedding_model = SentenceTransformer(model_name)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [6]:
answer_llm = df.iloc[0].answer_llm

In [42]:
v = embedding_model.encode(answer_llm)

In [43]:
v

array([-4.22446549e-01, -2.24856257e-01, -3.24058414e-01, -2.84758478e-01,
        7.25642918e-03,  1.01186566e-01,  1.03716910e-01, -1.89983174e-01,
       -2.80599259e-02,  2.71588802e-01, -1.15337655e-01,  1.14666030e-01,
       -8.49586725e-02,  3.32365334e-01,  5.52720726e-02, -2.22195774e-01,
       -1.42540857e-01,  1.02519155e-01, -1.52333647e-01, -2.02912465e-01,
        1.98422875e-02,  8.38149190e-02, -5.68632066e-01,  2.32844148e-02,
       -1.67292684e-01, -2.39256918e-01, -8.05464387e-02,  2.57084146e-02,
       -8.15464780e-02, -7.39290118e-02, -2.61550009e-01,  1.92575473e-02,
        3.22909206e-01,  1.90357104e-01, -9.34726413e-05, -2.13165611e-01,
        2.88943425e-02, -1.79530401e-02, -5.92756271e-02,  1.99918285e-01,
       -4.75170948e-02,  1.71634093e-01, -2.45917086e-02, -9.38061550e-02,
       -3.57002735e-01,  1.33263692e-01,  1.94045901e-01, -1.18530318e-01,
        4.56915230e-01,  1.47728190e-01,  3.35945129e-01, -1.86959356e-01,
        2.45954901e-01, -

In [28]:
# Q2

In [44]:
df.columns

Index(['answer_llm', 'answer_orig', 'document', 'question', 'course'], dtype='object')

In [27]:
v_llm  =[embedding_model.encode(v)  for v in df.answer_llm]

In [31]:
v_orig   =[embedding_model.encode(v)  for v in df.answer_orig ]

In [38]:
v_llm[0].dot(v_orig[0])

17.515987

In [45]:
# Create embeddings for the answer_llm and answer_orig columns
embeddings_llm = embedding_model.encode(df['answer_llm'].tolist())
embeddings_orig = embedding_model.encode(df['answer_orig'].tolist())

In [46]:
dot_products = [np.dot(embeddings_llm[i], embeddings_orig[i]) for i in range(len(embeddings_llm))]

In [47]:
np.percentile(dot_products, 75)

31.67430877685547

In [ ]:
# Q3

In [48]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

# Normalize the embeddings
embeddings_llm_norm = np.array([normalize(v) for v in embeddings_llm])
embeddings_orig_norm = np.array([normalize(v) for v in embeddings_orig])

# Compute cosine similarities and store them in a list
cosine_similarities = [np.dot(embeddings_llm_norm[i], embeddings_orig_norm[i]) for i in range(len(embeddings_llm_norm))]

# Calculate the 75th percentile of the cosine similarities
percentile_75_cosine = np.percentile(cosine_similarities, 75)

print("75th percentile of the cosine similarity scores:", percentile_75_cosine)

75th percentile of the cosine similarity scores: 0.8362348824739456


In [ ]:
#Q4

In [50]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [53]:
df.iloc[9]

answer_llm     You can access the pre-recorded course videos ...
answer_orig    The course videos are pre-recorded, you can st...
document                                                39fda9f0
question       Where can I access the pre-recorded course vid...
course                                 machine-learning-zoomcamp
Name: 9, dtype: object

In [54]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df.iloc[9]
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [55]:
scores

{'rouge-1': {'r': 0.2894736842105263,
  'p': 0.4230769230769231,
  'f': 0.34374999517578125},
 'rouge-2': {'r': 0.18, 'p': 0.2647058823529412, 'f': 0.21428570946712028},
 'rouge-l': {'r': 0.2894736842105263,
  'p': 0.4230769230769231,
  'f': 0.34374999517578125}}

In [57]:
# Extract the F1 scores for ROUGE-1, ROUGE-2, and ROUGE-L
f1_rouge_1 = scores['rouge-1']['f']
f1_rouge_2 = scores['rouge-2']['f']
f1_rouge_l = scores['rouge-l']['f']

# Calculate the average F1 score

average_f1= (f1_rouge_1 + f1_rouge_2 + f1_rouge_l) / 3
average_f1

0.3005952332728943